In [1]:
import pandas as pd
import datetime
import mariadb
import numpy as np
import mariadb
from pathlib import Path

In [2]:
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="jrynning",
        password="!cyrIOiKhb3",
        host="ajc-secure-datawarehouse.cgmwsizvte0i.us-east-1.rds.amazonaws.com",
        port=3306
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [3]:
csv_path = Path().cwd().parent.joinpath('voter_history_files/1-5-2021 - JANUARY 5, 2021 FEDERAL RUNOFF ELECTION.csv')
import_csv = pd.read_csv(csv_path, converters={'County Name': str, 'Voter Registration Number': str, 'Election Date':str, 'Election Type':str, 'Party': str, 'Ballot Style':str, 'Absentee':str, 'Provisional':str, 'Supplemental':str})

In [4]:
general_1_5_2021 = import_csv.copy()

In [5]:
general_1_5_2021

,County Name,Voter Registration Number,Election Date,Election Type,Party,Ballot Style,Absentee,Provisional,Supplemental
0,APPLING,00047002,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
1,APPLING,00122112,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
2,APPLING,00170146,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
3,APPLING,00174479,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
4,APPLING,00308860,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
...,...,...,...,...,...,...,...,...,...
4454245,MCDUFFIE,01458083,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
4454246,COBB,04058688,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
4454247,BRANTLEY,05070231,01/05/2021,GENERAL ELECTION RUNOFF,,MAIL IN,Y,N,N
4454248,LOWNDES,12885667,01/05/2021,GENERAL ELECTION RUNOFF,,MAIL IN,Y,N,N


In [6]:
df = general_1_5_2021.copy()

In [7]:
df = df.rename(columns={'County Name': 'county_name', 'Voter Registration Number': 'voter_registration_number', 'Election Date': 'election_date', 'Election Type': 'election_type', 'Party': 'party', 'Ballot Style': 'ballot_style', 'Absentee': 'absentee', 'Provisional': 'provisional', 'Supplemental':'supplemental'})

In [8]:
def add_row_names(column):
    counter = 0
    row_name_list = []
    for row in column:
        counter += 1
        row_name_list.append(counter)
    return row_name_list

In [9]:
df.insert(0, 'row_names', add_row_names(df['county_name']))

In [10]:
def column_float_to_strings(column):
    string_list = []
    for entry in column:
        if str(entry) == 'nan':
            string_list.append("nan")
        else:
            string_list.append(str(entry))
    return string_list

In [11]:
def remove_last_two_digits(column):
    string_list = []
    for entry in column:
        string_list.append(entry[0:-2])
    return string_list

In [12]:
df['voter_registration_number'][df['voter_registration_number'].isnull()]

Series([], Name: voter_registration_number, dtype: object)

In [13]:
df

,row_names,county_name,voter_registration_number,election_date,election_type,party,ballot_style,absentee,provisional,supplemental
0,1,APPLING,00047002,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
1,2,APPLING,00122112,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
2,3,APPLING,00170146,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
3,4,APPLING,00174479,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
4,5,APPLING,00308860,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
...,...,...,...,...,...,...,...,...,...,...
4454245,4454246,MCDUFFIE,01458083,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
4454246,4454247,COBB,04058688,01/05/2021,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
4454247,4454248,BRANTLEY,05070231,01/05/2021,GENERAL ELECTION RUNOFF,,MAIL IN,Y,N,N
4454248,4454249,LOWNDES,12885667,01/05/2021,GENERAL ELECTION RUNOFF,,MAIL IN,Y,N,N


In [14]:
def reformat_date(column):
    date_list = []
    for entry in column:
        date_list.append(f"{entry[6:10]}" + "-" + f"{entry[0:2]}" + "-" + f"{entry[3:5]}")
    return date_list

In [15]:
df['election_date'] = reformat_date(df['election_date'])

In [16]:
df

,row_names,county_name,voter_registration_number,election_date,election_type,party,ballot_style,absentee,provisional,supplemental
0,1,APPLING,00047002,2021-01-05,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
1,2,APPLING,00122112,2021-01-05,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
2,3,APPLING,00170146,2021-01-05,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
3,4,APPLING,00174479,2021-01-05,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
4,5,APPLING,00308860,2021-01-05,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
...,...,...,...,...,...,...,...,...,...,...
4454245,4454246,MCDUFFIE,01458083,2021-01-05,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
4454246,4454247,COBB,04058688,2021-01-05,GENERAL ELECTION RUNOFF,,REGULAR,N,N,N
4454247,4454248,BRANTLEY,05070231,2021-01-05,GENERAL ELECTION RUNOFF,,MAIL IN,Y,N,N
4454248,4454249,LOWNDES,12885667,2021-01-05,GENERAL ELECTION RUNOFF,,MAIL IN,Y,N,N


In [18]:
df.value_counts('election_type')

election_type
GENERAL ELECTION RUNOFF    4454250
Name: count, dtype: int64

In [19]:
df.value_counts('election_date')

election_date
2021-01-05    4454250
Name: count, dtype: int64

In [17]:
output_path = Path().cwd().parent.joinpath('output_csv/senate_runoff_2021.csv')
df.to_csv(output_path)